In [1]:
!pip install opensearch-py

     |████████████████████████████████| 266kB 9.9MB/s eta 0:00:01
     |████████████████████████████████| 163kB 54.7MB/s eta 0:00:01
     |████████████████████████████████| 153kB 55.6MB/s eta 0:00:01
     |████████████████████████████████| 235kB 51.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 10.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 16.6MB/s eta 0:00:01
     |████████████████████████████████| 143kB 57.7MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jsonschema-path 0.3.2 has requirement referencing<0.32.0,>=0.28.0, but you'll have referencing 0.33.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from opensearchpy import OpenSearch
host = 'vpc-aw-fdc-opensearch-qa-01-klbmt4ru7oksvsemnnmjpohfw4.us-east-1.es.amazonaws.com'
port = 443
auth = ('qa-developer', 'wF5341W8=~') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [2]:
query = {
  "size" : 10000,
  "sort" : [
        { "@timestamp" : {"order" : "desc"}}
  ],
  "query": {
    "bool": {
        "must": [
            {"match_phrase" : {"kubernetes.labels.app": "fosfor-gatekeeper"}},
            {"regexp": {"message": "[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}"}}
        ],
        "filter" : { "bool" :   {   
            "must_not": [
                {"regexp" : {"message": "elb"}},
                {"regexp" : {"message": "gatewayconfig"}},
            ]
            }
        }
      }
    }
}
    
response = client.search(
    body = query,
    index = 'filebeat-qa-*'
)

#response

In [3]:
import sys

count=0
for hit in response['hits']['hits']:
    count=count+1
    print(hit["_source"]["message"])
    if count > 10:
        break


10.229.12.241 - - [24/May/2024:12:50:16 +0000] "GET /sidecar/api/v1/nb-c5ade986-4ae7-412f-be0a-51cf7159109b-shivanand10639513ltim/status HTTP/1.1" 200 401 "https://qa.fdc.leni.ai/fdc/?tenantName=default-realm&userName=shivanand.10639513%40ltimindtree.com" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36" "15.206.245.70"
10.229.12.39 - - [24/May/2024:12:50:16 +0000] "POST /templates/12519f11689e0543823c8fafd90d1d66/c5ade986-4ae7-412f-be0a-51cf7159109b/Jupyterlab-3.9-cc2233a4-1b03-42ca-8c9a-185b464cdaee/api/kernels/d2b03f82-37ff-4a27-8bab-df04679c6ce9/restart?1716555016087 HTTP/1.1" 200 158 "https://qa.fdc.leni.ai/templates/12519f11689e0543823c8fafd90d1d66/c5ade986-4ae7-412f-be0a-51cf7159109b/Jupyterlab-3.9-cc2233a4-1b03-42ca-8c9a-185b464cdaee/lab/tree/Analysis/Gatekeeper_log.ipynb" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36" "15.206.245.70"
20

In [4]:
import sys

l_arr = []
for hit in response['hits']['hits']:
    s_arr = []
    str = hit["_source"]["message"].split()
    if len(str) > 9:
        if s_arr[6].endswith(
        s_arr.append(str[6])
        s_arr.append(str[8])
        s_arr.append(str[9])
        l_arr.append(s_arr)

import pandas as pd
df = pd.DataFrame(l_arr, columns=['URL', 'CODE', 'TIME'])

df["TIME"] = pd.to_numeric(df['TIME'], errors='coerce')
df.dropna(subset=['TIME'], inplace=True)
df.sort_values(by='TIME', inplace=True, ascending=False)

<ipython-input-4-6cc623a36ce9>:13: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [13]:
df[0:110].head(300)

,URL,CODE,TIME
3852,/templates/73a135196c0101af75b5e17bfe8b3a06/48...,200,25871264.0
6888,/pipeline/vendors-node_modules_date-io_date-fn...,200,18603544.0
841,/fdc/578.0617b3e1dc9658b6.bundle.js,200,6975263.0
902,/fdc/578.0617b3e1dc9658b6.bundle.js,200,6975263.0
1635,/fdc/578.0617b3e1dc9658b6.bundle.js,200,6975263.0
...,...,...,...
7681,/fdc/d049f3e08b20596b.otf,200,126280.0
7888,/fdc/d049f3e08b20596b.otf,200,126280.0
2675,/fdc/d049f3e08b20596b.otf,200,126280.0
8410,/fdc/d049f3e08b20596b.otf,200,126280.0


In [11]:
df2 = df[df["TIME"] > 60000]

In [12]:
df2["URL"].unique()

array(['/templates/73a135196c0101af75b5e17bfe8b3a06/48536631-7eb9-4db7-b55d-d88696a9c9ba/Jupyterlab-3.10-251bd3f0-88d6-4b30-a488-a4bfb5c5ab18/api/contents/Dynamic%20Pricing%20/data%20generation/final_data.csv?type=file&content=1&format=text&1716553696797',
       '/pipeline/vendors-node_modules_date-io_date-fns_build_index_esm_js-node_modules_material-ui_core_esm_Bo-f30ba0.74625b7a8bbafcf8.bundle.js',
       '/fdc/578.0617b3e1dc9658b6.bundle.js',
       '/fdc/578.515f3538683840c3.bundle.js',
       '/5326.209fe5f2254e395d.bundle.js',
       '/job/755.1ab44045d7c372c6.bundle.js',
       '/pipeline/src_lib_containers_flowdesign_index_js.663365b4230432da.bundle.js',
       '/pipeline/vendors-node_modules_material-ui_core_colors_green_js-node_modules_material-ui_icons_Add_js-n-d1df88.4aaba4de31d72943.bundle.js',
       '/templates/12519f11689e0543823c8fafd90d1d66/c5ade986-4ae7-412f-be0a-51cf7159109b/Jupyterlab-3.9-cc2233a4-1b03-42ca-8c9a-185b464cdaee/static/lab/jlab_core.8254cd1e48acf4f1b1

In [10]:
df_fail = df[df["CODE"] == "500"]
df_fail["URL"].unique()

array(['/project-manager/mosaic-console-backend/secured/api/project/v1/48c3bab2-f73f-4074-9aaa-3fe4cf6e80f9',
       '/project-manager/mosaic-console-backend/secured/api/project/v1/e2a70adf-a99e-4fcc-a564-1aa8065d5505',
       '/project-manager/mosaic-console-backend/secured/api/project/v1/1d2b4c37-d11a-4030-bff1-1705165909c0',
       '/project-manager/mosaic-console-backend/secured/api/project/v1/75dca0a1-567e-49ee-931a-95954783405f',
       '/project-manager/mosaic-console-backend/secured/api/project/v1/4d2126dd-7cb2-4fbc-8ec8-b6db3fd58da0',
       '/refract/mlflow/ajax-api/2.0/mlflow/experiments/search?max_results=50000&filter=tags.project_id=829b0193-234f-40ee-af67-319afdc129a8&order_by=last_update_time&view_type=ACTIVE_ONLY',
       '/refract/mlflow/ajax-api/2.0/mlflow/experiments/search?max_results=50000&filter=tags.project_id=e30bdc81-22af-4b72-92ae-1a6783b69344&order_by=last_update_time&view_type=ACTIVE_ONLY',
       '/project-manager/mosaic-console-backend/secured/api/project/